In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Подготовка датасета

In [3]:
df1 = pd.read_csv('dataset1.csv', encoding='cp1251', sep='\t', header=None)
df2 = pd.read_csv('dataset2.csv', encoding='cp1251', sep='\t', header=None, quoting=3)
df = pd.concat([df1, df2], axis=0).reset_index()
del df['index']

df.head()

,0,1,2,3,4,5,6,7,8
0,B057586722045,_ЁЭффективное численное решение задачи Стокса ...,_ЁРассматривается задача Стокса с разрывным ко...,_ЁХабаровск\_Ёзадача Стокса с разрывным коэффи...,e8\f2,13Д\16Б,27.41.19\30.17.02,271.41.19.17.21\301.17.02.11,###
1,B10213980294,_ЁАвтоматическая маршрутизация в среде с препя...,_ЁВ работе рассматривается перемещение тела гр...,_Ёавтоматическая маршрутизация\_Ёинтеллектуаль...,e8\e1\e9\00,93\93-Y\37\37-Y\РС\81,28.23.27\50.41.21\55.30.31,282.23.27.09\509.41.21.25.19\551.30.31.07.13,###
2,B103249601493,Структурно-динамический анализ с помощью ЯМР-р...,Изучение движения молекул и атомов в твердых т...,Татарстан\ЯМР-релаксометрия\газы природные\доб...,e5\f7,08Д\19А,31.01.33\38.57.23,311.01.33\383.57.23.01,###
3,B10349920193X,Структурно-функциональные исследования пептидо...,Излагается разрабатываемая концепция создания ...,изучение\лекарственные средства\пептиды\разраб...,e3,04Р1\04Б3,62.13.39,621.13.39,###
4,B103499202650,Применение хроматографии и масс-спектрометрии ...,В результате проведенных исследований разработ...,Galleria mellonella\антибактериальная активнос...,e3,04Р1\04Б3,62.13.39,621.13.39,###


In [48]:
df1[:50].to_csv('dataset1_example.csv', index=False)
df2[:50].to_csv('dataset2_example.csv', index=False)

In [4]:
txt = open('viniti_md.txt','r',encoding='cp1251',)
tx = txt.readlines()
alphabet = []
for x in tx:
    alphabet.append(x.strip('\n').split('\t'))
    
alpha_df = pd.DataFrame(alphabet, columns=['символ',"значение", "код", "категория"])

def valid_alpha(grp:list) -> list:
    unvalid_alpha = alpha_df.copy()
    unvalid_alpha.drop(unvalid_alpha.index[unvalid_alpha['категория'].isin(grp)], inplace=True)
    return sorted(unvalid_alpha['символ'], key=lambda x: len(x), reverse=True)

unvalid_alpha = valid_alpha(['буквы лат.','цифры','буквы рус.', 'пробел'])
for allowed_el in ['-', '.', '!', ',', ':', ';', '?']:
    unvalid_alpha.remove(allowed_el)

In [5]:
def text_processing(unvalid_alpha: list, s: str)->str:
    current_str = s
    for x in unvalid_alpha:
        current_str = current_str.replace(x, ' ')
    tokens = current_str.split(' ')
    current_str = ' '.join([token for token in tokens if token != ''])
        
    return current_str

In [6]:
%%time
name = 1
anot = 2
keywords = 3

df['keywords'] = df[keywords].apply(lambda x: list(map(lambda y: text_processing(unvalid_alpha, y), x.split('\\'))))
df['name'] = df[name].apply(lambda x: text_processing(unvalid_alpha, x))
df['anot'] = df[anot].apply(lambda x: text_processing(unvalid_alpha, x))

df_cleared = df[['keywords', 'name', 'anot']]
df_cleared['name_anot'] = df_cleared.apply(lambda x: x[1]+'. '+ x[2], axis=1)
df_cleared.head()

Wall time: 52.1 s


<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,keywords,name,anot,name_anot
0,"[Хабаровск, задача Стокса с разрывным коэффици...",Эффективное численное решение задачи Стокса с ...,Рассматривается задача Стокса с разрывным коэф...,Эффективное численное решение задачи Стокса с ...
1,"[автоматическая маршрутизация, интеллектуальны...",Автоматическая маршрутизация в среде с препятс...,В работе рассматривается перемещение тела груп...,Автоматическая маршрутизация в среде с препятс...
2,"[Татарстан, ЯМР-релаксометрия, газы природные,...",Структурно-динамический анализ с помощью ЯМР-р...,Изучение движения молекул и атомов в твердых т...,Структурно-динамический анализ с помощью ЯМР-р...
3,"[изучение, лекарственные средства, пептиды, ра...",Структурно-функциональные исследования пептидо...,Излагается разрабатываемая концепция создания ...,Структурно-функциональные исследования пептидо...
4,"[Galleria mellonella, антибактериальная активн...",Применение хроматографии и масс-спектрометрии ...,В результате проведенных исследований разработ...,Применение хроматографии и масс-спектрометрии ...


In [7]:
# Выборка из 10тысяч, по которым происходил замер качества BERT-моделей
df_exp = df_cleared.sample(10000, random_state=42).reset_index()
df_exp.to_csv('df_exp.csv', index=False)

# Выборка из 50 сэмплов, для использования в данном ноутбуке
df_exp_short = df_cleared.sample(50, random_state=42).reset_index()
df_exp_short.to_csv('df_exp_short.csv', index=False)

## Получение ключевых слов

In [8]:
def get_stopwords_list(add_external=True):
    nltk_list = nltk.corpus.stopwords.words('russian') 
    with open('stopwords-ru.txt', 'r', encoding='utf-8') as f:
        external_list = f.read().split('\n')
    final_list = list(set(nltk_list) | set(external_list)) if add_external == True else nltk_list
        
    return final_list

russian_stopwords = get_stopwords_list()

In [9]:
def predict_keyphrases(text, model, n_words):
    if model == 'RAKE':
        rake.extract_keywords_from_text(text)
        keyphrases = rake.get_ranked_phrases()
        
    if model == 'YAKE':
        keyphrases = list(map(lambda x: x[0], yake_model.extract_keywords(text)))
        
    if model == 'TextRank':
        cleaned_text = ' '.join([word.replace('\n', '') for word in text.split(' ') if word.lower() not in russian_stopwords])
        keyphrases = keywords.keywords(text, language = "russian").split("\n")
        
    if model == 'BERT_spacy':
        vectorizer = KeyphraseCountVectorizer(spacy_pipeline="ru_core_news_sm",
                                              stop_words=russian_stopwords,
                                              pos_pattern='<ADJ.*>*<N.*>+')
        bert_keywords = bert_kw.extract_keywords(docs=text,
                                                  vectorizer=vectorizer,
                                                  nr_candidates=20,
                                                  top_n=10,
                                                  use_mmr=True,
                                                  diversity=0.3,
                                                 )
        keyphrases = list(map(lambda x: x[0], bert_keywords))
        
    if model == 'BERT_sklearn':
        bert_keywords = bert_kw.extract_keywords(docs=text,
                                                  keyphrase_ngram_range=(1,3),
                                                  stop_words=russian_stopwords,
                                                  nr_candidates=20,
                                                  top_n=10,
                                                  use_mmr=True,
                                                  diversity=0.3)
        keyphrases = list(map(lambda x: x[0], bert_keywords))
        
    if model == 'YaKeyBert':
        keyphrases = YaKeyBert_PIPE(text, n_words)
    
    return keyphrases[:n_words]

### Rake

In [11]:
%%time
from rake_nltk import Rake

rake = Rake(stopwords=russian_stopwords, max_length=3)
df_exp_short['keyw_RAKE'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'RAKE', 10))

Wall time: 36.4 ms


### Yake

In [12]:
%%time
import yake

yake_model = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 3,          # максимальное количество слов в фразе
    dedupLim = 0.3, # порог похожести слов
    top = 10        # количество ключевых слов
)

df_exp_short['keyw_YAKE'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'YAKE', 10))

Wall time: 5.41 s


### TextRank

In [13]:
%time
from summa import keywords

df_exp_short['keyw_TextRank'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'TextRank', 10))

Wall time: 0 ns


### KeyBert

In [14]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

In [33]:
bert_kw = KeyBERT()

In [34]:
%%time
df_exp_short['keyw_BERT_spacy'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_spacy', 10))

Wall time: 1min 41s


In [35]:
%%time
df_exp_short['keyw_BERT_sklearn'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'BERT_sklearn', 10))

C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anacon

C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(
C:\Users\Ilya\anacon

Wall time: 1min 6s


### YaKeyBert

In [16]:
yake_model = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 4,          # максимальное количество слов в фразе
    dedupLim = 0.3, # порог похожести слов
    top = 10        # количество ключевых слов
)

vectorizer = KeyphraseCountVectorizer(spacy_pipeline="ru_core_news_sm",
                                      stop_words=russian_stopwords,
                                      pos_pattern='<ADJ.*>*<N.*>+')
hf_model = pipeline("feature-extraction", model='xlm-roberta-base')
bert_kw = KeyBERT(model=hf_model)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [20]:
def YaKeyBert_PIPE(text, n_keywords):
    bert_keywords = []
    
    yake_keyphrases = list(map(lambda x: x[0], yake_model.extract_keywords(text)))
    for phrase in yake_keyphrases:
        try:
            phrase_keywords = bert_kw.extract_keywords(
                                    docs=phrase,
                                    vectorizer=vectorizer,
                                    nr_candidates=20,
                                    top_n=1,
                                    use_mmr=True,
                                    diversity=0.3,)[0][0]
            n_words = len(phrase_keywords.split(' ')) 
            if n_words >= 1 and n_words <= 4:
                bert_keywords.append(phrase_keywords)
        except:
            pass
        
    keywords = list(set(bert_keywords))[:n_keywords]
        
    return keywords

In [21]:
%%time
df_exp_short['keyw_yakeybert_anot_10'] = df_exp_short['anot'].apply(lambda x: predict_keyphrases(x, 'YaKeyBert', 10))

Wall time: 13min 44s


In [22]:
%%time
df_exp_short['keyw_yakeybert_name_anot_10'] = df_exp_short['name_anot'].apply(lambda x: predict_keyphrases(x, 'YaKeyBert', 10))

Wall time: 12min 55s


## Оценка качества моделей

In [ ]:
from nltk.stem.snowball import SnowballStemmer

def stem_phrase(phrase):
    stemmer = SnowballStemmer("russian")  
    words = phrase.lower().split(' ')
    stemmed_phrase = ' '.join([stemmer.stem(word) for word in words])
    
    return stemmed_phrase

def check_occurence(check_phrase, phrases_list):
    for phrase in phrases_list:
        k = 0
        phrase_granulated = set(phrase.split(' '))
        check_phrase_granulated = check_phrase.split(' ')
        for check_word in check_phrase_granulated:
            if check_word in phrase_granulated:
                k += 1
        if k/len(check_phrase_granulated) > 0.5:
            return True
    
    return False
    
def keywords_precision_score(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
        
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for keyw in keyw_pred:
        if check_occurence(keyw, keyw_true): 
            n+=1
    
    return n/k

def keywords_recall_score(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    n_match = 0
    n_true = len(keyw_true)
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for keyw in keyw_true:
        if check_occurence(keyw, keyw_pred): 
            n_match += 1
    
    return n_match/n_true

def keywords_mean_reciprocal_rank(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for n_el, keyw in enumerate(keyw_pred):
        if check_occurence(keyw, keyw_true): 
            n = n_el + 1
            break
    
    return 1/n if n != 0 else 0

def keywords_mean_average_precision(keyw_true, keyw_pred, k):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
    
    idxs = []
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
        
    for n_el, keyw in enumerate(keyw_pred):
        if check_occurence(keyw, keyw_true):
            idxs.append(n_el + 1)
            
    if len(idxs) == 0:
        return 0
    
    precisions = [(n_el+1)/ idx for n_el, idx in enumerate(idxs)]
    return sum(precisions)/len(precisions)

def keywords_IoU(keyw_true, keyw_pred, k=5):
    if len(keyw_true) == 0 or len(keyw_pred) == 0:
        return 0
        
    n_intersect = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
    union = len(set(keyw_true + keyw_pred))
        
    for keyw in keyw_pred:
        if check_occurence(keyw, keyw_true): 
            n_intersect += 1
    
    return n_intersect/union

def keywords_stem(keyw_true, keyw_pred, k=5):
    n = 0
    keyw_true = [stem_phrase(phrase) for phrase in keyw_true]
    keyw_pred = [stem_phrase(phrase) for phrase in keyw_pred[:k]]
    
    return keyw_true, keyw_pred

def evaluate_extractor(y_true, y_pred, metric, k=5):
    metric_values = [] 
    
    try:
        if metric == 'precision':
            for idx in range(len(y_true)):
                value = keywords_precision_score(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'recall':
            for idx in range(len(y_true)):
                value = keywords_recall_score(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'MRR':
            for idx in range(len(y_true)):
                value = keywords_mean_reciprocal_rank(y_true[idx], y_pred[idx], k)
                metric_values.append(value)

        if metric == 'MAP':
            for idx in range(len(y_true)):
                value = keywords_mean_average_precision(y_true[idx], y_pred[idx], k)
                metric_values.append(value)
                
        if metric == 'IoU':
            for idx in range(len(y_true)):
                value = keywords_IoU(y_true[idx], y_pred[idx], k)
                metric_values.append(value)
            
        return np.mean(metric_values)
    
    except KeyError:
        print(y_true[idx])
        
def extraction_evaluation_report(df, keyw_true_col, models, metrics, size=0.1):
    result = dict()
    df = df.sample(round(size*len(df))).reset_index()
    
    for model_name in models:
        pred_column = 'keyw_' + model_name
        metrics_values = dict()
        for metric_name in metrics:
            metrics_values[metric_name+'_at_5'] = evaluate_extractor(df[keyw_true_col], df[pred_column], metric_name, k=5)
            metrics_values[metric_name+'_at_10'] = evaluate_extractor(df[keyw_true_col], df[pred_column], metric_name, k=10)
        
        result[model_name] = metrics_values
        
    return result

In [43]:
%%time
eval_result = extraction_evaluation_report(df_exp_short,
                                           'keywords',
                                           ['RAKE', 'YAKE', 'TextRank',
                                            'BERT_spacy', 'BERT_sklearn',
                                            'yakeybert_name_anot_10', 'yakeybert_anot_10'
                                           ],
                                           ['precision', 'recall', 'MRR', 'MAP', 'IoU'],
                                           size=1)

Wall time: 4.18 s


In [44]:
for model in eval_result.keys():
    print("Extractor:", model)
    for metric in eval_result[model].keys():
        print('\t', metric, ":", round(eval_result[model][metric], 3))

Extractor: RAKE
	 precision_at_5 : 0.1
	 precision_at_10 : 0.074
	 recall_at_5 : 0.165
	 recall_at_10 : 0.252
	 MRR_at_5 : 0.175
	 MRR_at_10 : 0.191
	 MAP_at_5 : 0.174
	 MAP_at_10 : 0.187
	 IoU_at_5 : 0.058
	 IoU_at_10 : 0.06
Extractor: YAKE
	 precision_at_5 : 0.244
	 precision_at_10 : 0.188
	 recall_at_5 : 0.339
	 recall_at_10 : 0.412
	 MRR_at_5 : 0.395
	 MRR_at_10 : 0.421
	 MAP_at_5 : 0.397
	 MAP_at_10 : 0.39
	 IoU_at_5 : 0.131
	 IoU_at_10 : 0.133
Extractor: TextRank
	 precision_at_5 : 0.292
	 precision_at_10 : 0.204
	 recall_at_5 : 0.103
	 recall_at_10 : 0.148
	 MRR_at_5 : 0.348
	 MRR_at_10 : 0.364
	 MAP_at_5 : 0.376
	 MAP_at_10 : 0.396
	 IoU_at_5 : 0.182
	 IoU_at_10 : 0.186
Extractor: BERT_spacy
	 precision_at_5 : 0.156
	 precision_at_10 : 0.124
	 recall_at_5 : 0.311
	 recall_at_10 : 0.415
	 MRR_at_5 : 0.284
	 MRR_at_10 : 0.299
	 MAP_at_5 : 0.282
	 MAP_at_10 : 0.274
	 IoU_at_5 : 0.088
	 IoU_at_10 : 0.092
Extractor: BERT_sklearn
	 precision_at_5 : 0.16
	 precision_at_10 : 0.128
	 re